# Artificial Neural Net Algorithm

This Algorithm mimics the an Artificial Neural Network Model. The algorithm has been implemented using the logics behind the working of an Artificial Neural Network. The Algorithm utilizes basic and prebuilt Python libraries to implement and simulate a 3 layer Artificial Neural Net. 

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
from numpy.random import RandomState as random_state_
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 


In [ ]:

import functions

We use drug_activity.csv file for training and testing the Model. The dataset is a Sparse Matrix containing 800 rows and 6062 columns, each row representing a certain molecule and each column representing a certain feature of the molecules. 

The first column of the dataset contains the labels for whether the molecule is active (1) or inactive (0) when it comes to making drugs. Active molecules are good for drug making. 

The dataset is imbalanced however; it contains only 78 Active molecules. 


In [ ]:
# importing dataset to Pandas dataframe
train_df = pd.read_csv('drug_activity.csv', sep='delimiter', header=None)

# separating comma separated values 
train_df = train_df[0].str.split(",", expand = True)

Preview Data

In [ ]:
# preview 10 random Molecules from the dataframe
train_df.sample(n=10)


Preview of Label Counts (Active and Inactive Molecules)

In [ ]:
# label value counts 
train_df[0].value_counts()


Some data preparations and cleaning

In [ ]:
# dropping Nan/None values
train_df = train_df.fillna(0)

# truncating dataframe. selecting only relevant features. dropping features with 0s
train_df = train_df.iloc[:,:500]

# convert all columns of DataFrame
train_df = train_df.apply(pd.to_numeric) 

Preview of new dataframe

In [ ]:
train_df.sample(n=10)


Selecting features and attributes (X and Y labels) for our Model

In [ ]:
# first prepare arrays from train data 
train_array = train_df.values

# features
features_ = train_array[:,1:]

# attributes
attributes_ = train_array[:,0]


###### Random Oversampling to handle the imbalanced data

In [ ]:
# library to use for Random Oversampling
from imblearn.over_sampling import RandomOverSampler as sm

# oversampling class object with random state
oversample = sm(random_state = 100)

# Oversampling the data with the object
features, attributes = oversample.fit_resample(features_,attributes_)


###### Data Splitting

In [ ]:
from sklearn.model_selection import train_test_split

# splitting data 
X_train, X_test, y_train, y_test = train_test_split(features, attributes, test_size=0.2)

###### Data Processing

In [ ]:
# convert y_train and y_test to array for effective calculations
y_train = np.array(y_train, dtype=np.int64, order='C')
y_test = np.asarray(y_test, dtype=np.int64, order='C')


In [ ]:
from sklearn import preprocessing

# preprocessing X_train and X_test
X_train = preprocessing.scale(X_train).T
X_test = preprocessing.scale(X_test).T


In [ ]:
# some initializations for calculations 
rs_ = random_state_(1234567890)
np.random.seed(1)


class algorithm:

    # initialize the model class objects
    def __init__(self, X_train, y_train, layers):
        
        self.layers = layers
        
        self.X = X_train
        self.Y = np.array(y_train).reshape(1, y_train.shape[0])
        
        self.L = len(layers) - 1
        
        self.cache_ = {}
        self.grads_ = {}
        self.params_ = {}
        
        layers[0] = (X_train.shape[0], None)
        
        k = len(layers)

        # initiliaze parameters
        for i in range(1, k):
            
            previous_layer = layers[i-1][0]
            current_layer = layers[i][0]
            
            self.params_['W' + str(i)] = rs_.randn(current_layer, previous_layer) * np.sqrt(2 / previous_layer)
            self.params_['b' + str(i)] = np.zeros((current_layer, 1))


    def _forward_(self):
        
        Z = self.params_['W1'].dot(self.X) + self.params_['b1']
        
        A = functions.activate(Z, layers[1][1])
        
        self.cache_['Z1'] = Z
        self.cache_['A1'] = A
         
        k = len(self.layers)
        
        for i in range(2, k):
            
            a_previous = A
            
            Z = self.params_['W' + str(i)].dot(a_previous) + self.params_['b' + str(i)]
            
            A = functions.activate(Z, layers[i][1])
            
            self.cache_['Z' + str(i)] = A
            self.cache_['A' + str(i)] = Z
            
            
         

    def _backward_(self, lambd):
        
        dA_ = 0
        
        A = self.cache_['A' + str(self.L)]
        
        if self.layers[self.L][1] == 'sigmoid': 
            
            dA_ = -np.divide(self.Y, A) - np.divide(1 - self.Y, 1 - A)
        
        k = self.L + 1
        
        for i in reversed(range(1, k)):
            
            a_previous = np.array([])
            
            scalar = self.X.shape[1]
            
            if i == 1:
                a_previous = self.X
                
            else : 
                a_previous = self.cache_['A' + str(i-1)]
                
            # calculate the gradients
            dZ_ = dA_ * functions.d_activate(self.cache_['Z' + str(i)], layers[i][1])
            dA_ = np.dot(self.params_['W' + str(i)].T, dZ_)
            
            # store gradients
            self.grads_['dW' + str(i)] = (1/scalar) * dZ_.dot(a_previous.T) + (lambd/scalar)*self.params_['W' + str(i)]
            self.grads_['db' + str(i)] = (1/scalar) * np.sum(dZ_, axis=1, keepdims=True)

            
            
    # update model parameters
    def _params_(self, alpha):
        
        k = self.L+1
        
        for i in range(1, k):
            
            self.params_['W' + str(i)] = self.params_['W' + str(i)] - alpha * self.grads_['dW' + str(i)]
            self.params_['b' + str(i)] = self.params_['b' + str(i)] - alpha * self.grads_['db' + str(i)]
            
            
            
    # calculate cost
    def _cost_(self, lambd):
        
        A = self.cache_['A' + str(self.L)]
        
        m = self.Y.shape[1]
        
        # regularization sum
        r_sum = 0
        
        k = len(self.layers) 
        
        for i in range(1, k):
            
            r_sum += np.sum(np.square(self.params_['W' + str(i)]))
    
        
        cost = - (1/m) * ( np.dot(self.Y, A.T) + np.dot(1 - self.Y, 1 - A.T)) + (lambd/(2*m)) * r_sum
        
        return np.squeeze(cost)   
            
    
    # train the algorithm
    def train(self, alpha, iterations, lambd, cost_display=False):
        
        costs = []
        
        for i in range(iterations):
            
            # calling forward function to iterate through layer
            self._forward_()
            
            # calling backward function
            self._backward_(lambd)
            
            # calling params function to update parameters
            self._params_(alpha)
            
            self._backward_(lambd)
            self._params_(alpha)
            
        # display cost 
        if cost_display == True:
            # calling costs function to calculate cost
            cost = self._cost_(lambd)
            print('-----------------------------------------------------------')
            print('Cost: --')
            print(f'{cost}')
                

            
                

    # function to handle predictions
    def predict(self, X_test):
        
        Z = self.params_['W1'].dot(X_test) + self.params_['b1']
        
        A = functions.activate(Z, layers[1][1])
        
        k = len(self.layers)
        
        for i in range(2,k):
            
            a_previous = A
            
            Z = self.params_['W' + str(i)].dot(a_previous) + self.params_['b' + str(i)]
            
            A = functions.activate(Z, layers[i][1])
            
        return np.where(A > .5, 1, 0)
    
    


## Model built from the Algorithm

### Model Initilization

In [ ]:
# initialize 3 layers for the Model
layers = {1: (5, 'relu'), 2: (15, 'relu'), 3: (1, 'sigmoid')}

# initializing the Model 
model = algorithm(X_train, y_train, layers)


### Model Training and Classification Predictions

Train Model from Algorithm

In [ ]:
# train model alpha value (.05), iterations (1000) and lambda value (1)

toc = time.time()
model.train(.005, 1000, 1, cost_display=True)
tic = time.time()

# time taken for training in seconds
print('-----------------------------------------------------------')
print(f'Time taken for training model (s): {tic - toc}')


Use Model for Prediction

In [ ]:
import time

# predictions from Model
toc = time.time()
y_pred = model.predict(X_test)
tic = time.time()

# time taken for prediction in seconds
print('-----------------------------------------------------------')
print(f'Time taken for prediction (s): {tic - toc}')


### Perfomance of the Model

In [ ]:
# compute perfomance scores
accuracy_score_ = accuracy_score(y_test, y_pred)
confusion_matrix_ = confusion_matrix(y_test, y_pred)

# Ouput Performance of Algorithm
print('-----------------------------------------------------------')
print('Perfomance of the Model in Drug Molecules Classification')
print('-----------------------------------------------------------')
print(f'Accuracy Score: {accuracy_score_}')
print('-----------------------------------------------------------')
print('Confusion Matrix:')
print('-----------------')
print(f'{confusion_matrix_}')
